In [2]:
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt

In [14]:
file_dir = r'C:/Users/shine/Desktop/BigData/Mentoring/0919'
save_file = r'C:/Users/shine/Desktop/BigData/Mentoring/0919/preprocess data/서울 20~30대 생활인구(6~8월).csv'
all_file = glob.glob(os.path.join(file_dir, 'LOCAL_PEOPLE_DONG_20200*.csv'))

In [15]:
people_data = []
for file in all_file:
    people = pd.read_csv(file, index_col=False)
    people = people[['기준일ID','행정동코드',
                     '남자20세부터24세생활인구수',
                     '남자25세부터29세생활인구수',
                     '남자30세부터34세생활인구수',
                     '남자35세부터39세생활인구수',
                     '여자20세부터24세생활인구수',
                     '여자25세부터29세생활인구수',
                     '여자30세부터34세생활인구수',
                     '여자35세부터39세생활인구수']]
    people_data.append(people)

In [16]:
# 한 df로 묶기
dataCombine = pd.concat(people_data, axis=0, ignore_index = True)
dataCombine = dataCombine.sort_values(by=['기준일ID','행정동코드'])
dataCombine = dataCombine.round(0)
dataCombine = dataCombine.astype(int, errors = 'ignore')
# 행정동코드 df와 결합을 위해 행정구 코드(5자리)로 수정
dataCombine = dataCombine.astype({'행정동코드':'str'})
dataCombine['행정동코드'] = dataCombine['행정동코드'].str[0:5]
dataCombine

,기준일ID,행정동코드,남자20세부터24세생활인구수,남자25세부터29세생활인구수,남자30세부터34세생활인구수,남자35세부터39세생활인구수,여자20세부터24세생활인구수,여자25세부터29세생활인구수,여자30세부터34세생활인구수,여자35세부터39세생활인구수
0,20200601,11110,612,502,396,566,421,419,451,617
761,20200601,11110,622,494,390,574,427,426,439,604
1097,20200601,11110,617,509,382,577,427,428,435,604
1272,20200601,11110,621,512,386,575,437,429,433,609
1769,20200601,11110,621,533,383,581,436,436,432,617
...,...,...,...,...,...,...,...,...,...,...
934129,20200831,11740,630,809,858,914,778,906,897,1150
934576,20200831,11740,656,845,922,990,793,912,961,1124
935242,20200831,11740,709,922,968,1033,808,962,943,1134
935433,20200831,11740,741,934,1023,1055,848,990,954,1171


In [17]:
code = pd.read_excel('행정동_20200814.xlsx')
seoul_code = code.loc[code['시도명'] == '서울특별시']
seoul_code = seoul_code.drop(['시도명','읍면동명'], axis=1)
seoul_code = seoul_code.dropna()
# slicing을 위해 행정동코드를 문자열로 바꾸고 10자리인 행정동코드를 slicing으로 8자리로 변경
seoul_code = seoul_code.astype(str)
seoul_code['행정동코드'] = seoul_code['행정동코드'].str[0:5]
seoul_code = seoul_code.drop_duplicates()
seoul_code

,행정동코드,시군구명
1,11110,종로구
19,11140,중구
35,11170,용산구
52,11200,성동구
70,11215,광진구
86,11230,동대문구
101,11260,중랑구
118,11290,성북구
139,11305,강북구
153,11320,도봉구


In [18]:
local_people = pd.merge(dataCombine, seoul_code, how='right', on='행정동코드')
local_people = local_people[['기준일ID','시군구명',
                             '남자20세부터24세생활인구수','남자25세부터29세생활인구수',
                             '남자30세부터34세생활인구수','남자35세부터39세생활인구수',
                             '여자20세부터24세생활인구수','여자25세부터29세생활인구수',
                             '여자30세부터34세생활인구수','여자35세부터39세생활인구수']]

local_people = local_people.rename(columns={'기준일ID':'날짜',
                                            '시군구명':'행정구',
                                            '남자20세부터24세생활인구수':'남20~24세',
                                            '남자25세부터29세생활인구수':'남25~29세',
                                            '남자30세부터34세생활인구수':'남30~34세',
                                            '남자35세부터39세생활인구수':'남35~39세',
                                            '여자20세부터24세생활인구수':'여20~24세',
                                            '여자25세부터29세생활인구수':'여25~29세',
                                            '여자30세부터34세생활인구수':'여30~34세',
                                            '여자35세부터39세생활인구수':'여35~39세'})
local_people = local_people.groupby(['날짜','행정구']).mean()
local_people = local_people.round(0)
local_people = local_people.astype(int)
local_people['생활인구'] = local_people.sum(axis=1)
local_people = local_people.drop(columns = {'남20~24세','남25~29세','남30~34세','남35~39세',
                                            '여20~24세','여25~29세','여30~34세','여35~39세'})
local_people = local_people.reset_index()
local_people

,날짜,행정구,생활인구
0,20200601,강남구,14292
1,20200601,강동구,7504
2,20200601,강북구,5988
3,20200601,강서구,8729
4,20200601,관악구,9399
...,...,...,...
2295,20200831,용산구,6252
2296,20200831,은평구,7804
2297,20200831,종로구,5360
2298,20200831,중구,6641


In [42]:
# 날짜 데이터프레임
date = pd.DataFrame(data = local_people['날짜'])
date = date.drop_duplicates()
date['날짜'] = pd.to_datetime(date['날짜'], format='%Y%m%d')
date = date.reset_index(drop=True)
date

,날짜
0,2020-06-01
1,2020-06-02
2,2020-06-03
3,2020-06-04
4,2020-06-05
...,...
87,2020-08-27
88,2020-08-28
89,2020-08-29
90,2020-08-30


In [44]:
district = pd.DataFrame(data = local_people[['행정구','생활인구']])

# 구별 생활인구 컬럼 만들기
gn = district[district['행정구'].isin(['강남구'])].reset_index()
gn = pd.DataFrame(data = gn['생활인구'])
gd = district[district['행정구'].isin(['강동구'])].reset_index()
gd = pd.DataFrame(data = gd['생활인구'])
gb = district[district['행정구'].isin(['강북구'])].reset_index()
gb = pd.DataFrame(data = gb['생활인구'])
gs = district[district['행정구'].isin(['강서구'])].reset_index()
gs = pd.DataFrame(data = gs['생활인구'])
ga = district[district['행정구'].isin(['관악구'])].reset_index()
ga = pd.DataFrame(data = ga['생활인구'])
gj = district[district['행정구'].isin(['광진구'])].reset_index()
gj = pd.DataFrame(data = gj['생활인구'])
gr = district[district['행정구'].isin(['구로구'])].reset_index()
gr = pd.DataFrame(data = gr['생활인구'])
gc = district[district['행정구'].isin(['금천구'])].reset_index()
gc = pd.DataFrame(data = gc['생활인구'])
nw = district[district['행정구'].isin(['노원구'])].reset_index()
nw = pd.DataFrame(data = nw['생활인구'])
db = district[district['행정구'].isin(['도봉구'])].reset_index()
db = pd.DataFrame(data = db['생활인구'])
ddm = district[district['행정구'].isin(['동대문구'])].reset_index()
ddm = pd.DataFrame(data = ddm['생활인구'])
dj = district[district['행정구'].isin(['동작구'])].reset_index()
dj = pd.DataFrame(data = dj['생활인구'])
mp = district[district['행정구'].isin(['마포구'])].reset_index()
mp = pd.DataFrame(data = mp['생활인구'])
sdm = district[district['행정구'].isin(['서대문구'])].reset_index()
sdm = pd.DataFrame(data = sdm['생활인구'])
sc = district[district['행정구'].isin(['서초구'])].reset_index()
sc = pd.DataFrame(data = sc['생활인구'])
sd = district[district['행정구'].isin(['성동구'])].reset_index()
sd = pd.DataFrame(data = sd['생활인구'])
sb = district[district['행정구'].isin(['성북구'])].reset_index()
sb = pd.DataFrame(data = sb['생활인구'])
sp = district[district['행정구'].isin(['송파구'])].reset_index()
sp = pd.DataFrame(data = sp['생활인구'])
yc = district[district['행정구'].isin(['양천구'])].reset_index()
yc = pd.DataFrame(data = yc['생활인구'])
ydp = district[district['행정구'].isin(['영등포구'])].reset_index()
ydp = pd.DataFrame(data = ydp['생활인구'])
ys = district[district['행정구'].isin(['용산구'])].reset_index()
ys = pd.DataFrame(data = ys['생활인구'])
ep = district[district['행정구'].isin(['은평구'])].reset_index()
ep = pd.DataFrame(data = ep['생활인구'])
jr1 = district[district['행정구'].isin(['종로구'])].reset_index()
jr1 = pd.DataFrame(data = jr1['생활인구'])
j = district[district['행정구'].isin(['중구'])].reset_index()
j = pd.DataFrame(data = j['생활인구'])
jr2 = district[district['행정구'].isin(['중랑구'])].reset_index()
jr2 = pd.DataFrame(data = jr2['생활인구'])

# 구별 데이터 합치기
district = pd.concat([gn,gd,gb,gs,ga,gj,gr,gc,nw,db,ddm,dj,mp,sdm,sc,sd,sb,sp,yc,ydp,ys,ep,jr1,j,jr2],
                     axis=1, ignore_index=True)
# 컬럼명 지정
district.columns = ['LP(강남)','LP(강동)','LP(강북)','LP(강서)','LP(관악)',
                    'LP(광진)','LP(구로)','LP(금천)','LP(노원)','LP(도봉)',
                    'LP(동대문)','LP(동작)','LP(마포)','LP(서대문)','LP(서초)',
                    'LP(성동)','LP(성북)','LP(송파)','LP(양천)','LP(영등포)',
                    'LP(용산)','LP(은평)','LP(종로)','LP(중구)','LP(중랑)']
# 날짜 붙이기
district = date.join(district).set_index('날짜')
# 맨 위에 save_file 변수에 지정한 파일 경로와 파일명으로 저장
district.to_csv(save_file, encoding = 'utf-8-sig')
district

,LP(강남),LP(강동),LP(강북),LP(강서),LP(관악),LP(광진),LP(구로),LP(금천),LP(노원),LP(도봉),...,LP(성동),LP(성북),LP(송파),LP(양천),LP(영등포),LP(용산),LP(은평),LP(종로),LP(중구),LP(중랑)
날짜,,,,,,,,,,,,,,,,,,,,,
2020-06-01,14292,7504,5988,8729,9399,8701,8227,8394,7083,5045,...,7155,6548,8668,5523,9690,6492,7522,6022,7237,6119
2020-06-02,14566,7488,5987,8696,9408,8675,8223,8415,7075,5023,...,7184,6563,8660,5530,9656,6500,7512,6177,7308,6117
2020-06-03,14649,7508,5964,8667,9366,8676,8171,8402,7047,5014,...,7190,6559,8681,5472,9674,6523,7470,6312,7328,6080
2020-06-04,14683,7455,5940,8650,9356,8675,8166,8364,7051,4995,...,7211,6533,8667,5464,9670,6560,7461,6298,7349,6075
2020-06-05,14487,7454,5924,8601,9322,8748,8111,8183,7009,4990,...,7182,6469,8686,5444,9680,6555,7441,6282,7225,6079
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08-27,13418,7630,6039,8844,9493,8570,8137,8224,6966,5087,...,7137,6351,8623,5432,9342,6266,7798,5399,6629,6244
2020-08-28,13402,7533,5934,8674,9301,8487,7984,8068,6869,5005,...,7074,6246,8511,5357,9252,6252,7661,5375,6595,6157
2020-08-29,10445,7927,6400,8893,9955,8813,7691,6881,7306,5373,...,6584,6507,8412,5631,8176,5910,8347,4280,4568,6712
